In [1]:
using Pkg

# LogRoller.jl: ローテーション可能なロギングを可能にするパッケージ
Pkg.add("LogRoller")

# HTTP.jl: 最新の master ブランチからインストール
Pkg.add(PackageSpec(name="HTTP", rev="master"))

    Updating registry at `C:\Users\user\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\user\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\user\.julia\environments\v1.7\Manifest.toml`
    Updating git-repo `https://github.com/JuliaWeb/HTTP.jl.git`
   Resolving package versions...
  No Changes to `C:\Users\user\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\user\.julia\environments\v1.7\Manifest.toml`


In [1]:
using HTTP, LogRoller, Logging, Sockets

"""
Global Logger: RollingLogger
- file name: "./WebSocket.log"
- size limit: 1MB (1,000,000 bytes)
- maintain rotated file count: 3
- log level: Debug
"""
# Set global logger.
global_logger(
    RollingLogger("./WebSocket.log", 1_000_000, 3, Logging.Debug)
)

"""
    createWebSocketServer(; hostname::String = "127.0.0.1", port::UInt16 = UInt16(8081))
        -> (server::IOServer, task::Task)

Create WebSocket Server.
"""
createWebSocketServer(; hostname::String = "127.0.0.1", port::UInt16 = UInt16(8081)) = begin
    server = Sockets.listen(Sockets.InetAddr(parse(IPAddr, hostname), port))
    task = @async HTTP.WebSockets.listen(hostname, port; server = server) do ws
        # タスク内のエラー処理は明示的に try-catch した方が良い
        try
            for msg in ws
                HTTP.WebSockets.send(ws, msg)
            end
        catch e
            @error "WebSocketServer Error" exception=(e, catch_backtrace())
            rethrow() # プロセスを終了するために rethrow する
        end
    end
    server, task
end

"""
    createWebSocketClient(endpoint::String)
        -> (task::Task, in_channel::Channel, out_channel::Channel)

Create WebSocket Client.

- channels:
    - in_channel:
        - type: `String`
        - size: 1
    - out_channel:
        - type `Any`
        - size: 1
- events:
    - in_channel: `exit`
        - Terminate the websocket client.
    - in_channel: `send:{any_string_data}`
        - Send `any_string_data` to the websocket server.
        - Response data will be written in out_channel: `Response: {any_string_data}`
    - in_channel: `exec:{function_name}`
        - Execute function to websocket stream: `function_name(ws::HTTP.Stream)`
        - Function return value will be written in out_channel.
"""
createWebSocketClient(endpoint::String) = begin
    c_in = Channel{String}(1)
    c_out = Channel(1)
    task = @async HTTP.WebSockets.open(endpoint) do ws
        # タスク内のエラー処理は明示的に try-catch した方が良い
        try
            # WebSocket の疎通が続く限りループする
            ## https://juliaweb.github.io/HTTP.jl/dev/reference/#HTTP.WebSockets.isclosed
            while !HTTP.WebSockets.isclosed(ws)
                command = take!(c_in) # wait until the in_channel is entered.
                commands = string.(split(command, ":", keepempty=false))

                @debug commands

                if commands[1] === "exit"
                    @info "Terminate websocket client..."
                    break
                elseif commands[1] === "send"
                    HTTP.WebSockets.send(ws, commands[2])
                    response = HTTP.WebSockets.receive(ws)
                    put!(c_out, string("Response: ", response))
                elseif commands[1] === "exec"
                    f = Symbol(commands[2])
                    output = @eval $f($ws)
                    @debug output
                    put!(c_out, output)
                end
            end
        catch e
            @error "WebSocketClient Error" exception=(e, catch_backtrace())
            showerror(stderr, e, catch_backtrace()) # コンソールにもエラー表示する
            rethrow() # プロセスを終了するために rethrow する
        end
    end
    task, c_in, c_out
end

createWebSocketClient

In [2]:
server, server_task = createWebSocketServer()

(Sockets.TCPServer(Base.Libc.WindowsRawSocket(0x0000000000000410) active), Task (runnable) @0x000000000de5e100)

In [3]:
client, c_in, c_out = createWebSocketClient("ws://127.0.0.1:8081")

(Task (runnable) @0x000000000f2b1180, Channel{String}(1), Channel{Any}(1))

In [4]:
istaskdone(server_task), istaskdone(client)

(false, false)

In [5]:
# Send data ("Hello WebSocket!!") to the WebSocket client.
put!(c_in, "send:Hello WebSocket!!")

"send:Hello WebSocket!!"

In [6]:
take!(c_out)

"Response: Hello WebSocket!!"

In [7]:
# Close WebSocket server.
close(server)

istaskdone(server_task), istaskdone(client)

(true, false)

In [8]:
# すでに閉じている WebSocket サーバにデータを送信した場合
put!(c_in, "send:Hello WebSocket!!")

"send:Hello WebSocket!!"

In [9]:
take!(c_out)

"Response: Hello WebSocket!!"

確認した感じ、WebSocket サーバが閉じても、クライアント側も同時に閉じるということはなさそう

そのため、クライアント側では定期的に Ping Pong 疎通確認をする必要がある

In [1]:
using HTTP, LogRoller, Logging, Sockets

"""
    createWebSocketServer(; hostname::String = "127.0.0.1", port::UInt16 = UInt16(8081))
        -> (server::IOServer, task::Task)

Create WebSocket Server.
"""
createWebSocketServer(; hostname::String = "127.0.0.1", port::UInt16 = UInt16(8081)) = begin
    server = Sockets.listen(Sockets.InetAddr(parse(IPAddr, hostname), port))
    task = @async HTTP.WebSockets.listen(hostname, port; server = server) do ws
        try
            for msg in ws
                HTTP.WebSockets.send(ws, msg)
            end
        catch e
            @error "WebSocketServer Error" exception=(e, catch_backtrace())
            rethrow() # プロセスを終了するために rethrow する
        end
    end
    server, task
end

"""
    createWebSocketClient(endpoint::String)
        -> (task::Task, in_channel::Channel, out_channel::Channel)

Create WebSocket Client.

- channels:
    - in_channel:
        - type: `String`
        - size: 1
    - out_channel:
        - type `Any`
        - size: 1
- events:
    - in_channel: `exit`
        - Terminate the websocket client.
    - in_channel: `send:{any_string_data}`
        - Send `any_string_data` to the websocket server.
        - Response data will be written in out_channel: `Response: {any_string_data}`
"""
createWebSocketClient(endpoint::String) = begin
    c_in = Channel{String}(1)
    c_out = Channel(1)
    task = @async HTTP.WebSockets.open(endpoint) do ws
        # タスク内でサブタスクを作成したい場合は @sync ブロック内で行う
        @sync begin
            # メインタスク
            maintask = @async try
                while !HTTP.WebSockets.isclosed(ws)
                    command = take!(c_in) # wait unitl command input
                    commands = string.(split(command, ":", keepempty=false))

                    @debug commands

                    if commands[1] === "exit"
                        @info "Terminate websocket client..."
                        break
                    elseif commands[1] === "send"
                        HTTP.WebSockets.send(ws, commands[2])
                        response = HTTP.WebSockets.receive(ws)
                        put!(c_out, string("Response: ", response))
                    elseif commands[1] === "exec"
                        f = Symbol(commands[2])
                        output = @eval $f($ws)
                        @debug output
                        put!(c_out, output)
                    end
                end
            catch e
                @error "WebSocketClient Error" exception=(e, catch_backtrace())
                showerror(stderr, e, catch_backtrace())
                rethrow()
            end

            # Ping Pong 疎通確認タスク
            pingtask = @async try
                while !HTTP.WebSockets.isclosed(ws)
                    HTTP.WebSockets.ping(ws)
                    HTTP.WebSockets.pong(ws)
                    sleep(1)
                end
            catch e
                @error "WebSocketClient Ping Error" exception=(e, catch_backtrace())
                showerror(stderr, e, catch_backtrace())
                rethrow()
            end
        end
    end
    task, c_in, c_out
end

createWebSocketClient

In [2]:
server, server_task = createWebSocketServer()
client, c_in, c_out = createWebSocketClient("ws://127.0.0.1:8081")
istaskdone(server_task), istaskdone(client)

(false, false)

In [3]:
# Send data ("Hello WebSocket!!") to the WebSocket client.
put!(c_in, "send:Hello WebSocket!!")
take!(c_out)

"Response: Hello WebSocket!!"

In [4]:
# Close WebSocket server.
close(server)

sleep(0.01)

istaskdone(server_task), istaskdone(client)

(true, false)

In [5]:
# WebSocket Client タスク強制終了
@async Base.throwto(client, InterruptException())

┌ Error: 2f3af80f-c6e2-4a51-818c-628eb580c335: error
│   (e, catch_backtrace()) = (InterruptException(), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00000000106415c3, Ptr{Nothing} @0x00000000106417c7, Ptr{Nothing} @0x00000000103c971e, Ptr{Nothing} @0x00000000103c984a, Ptr{Nothing} @0x00000000109a3840, Ptr{Nothing} @0x00000000109adff4, Ptr{Nothing} @0x000000001cbdbe8a, Ptr{Nothing} @0x000000001cbe1e8b, Ptr{Nothing} @0x000000001cbe3e01, Ptr{Nothing} @0x000000001cbe4da9, Ptr{Nothing} @0x000000001cbe5445, Ptr{Nothing} @0x000000001cbe5c9f, Ptr{Nothing} @0x000000001cbe66a9, Ptr{Nothing} @0x000000001cbcca08, Ptr{Nothing} @0x000000001cbcda12, Ptr{Nothing} @0x000000001cbce451, Ptr{Nothing} @0x000000001cbcf35b, Ptr{Nothing} @0x000000001cbd00ef, Ptr{Nothing} @0x000000001cbd06ae, Ptr{Nothing} @0x000000001cbd0881, Ptr{Nothing} @0x000000001cbd08f4, Ptr{Nothing} @0x000000001cbb193d, Ptr{Nothing} @0x000000001cbb1ac8, Ptr{Nothing} @0x000000001cbacb2f, Ptr{Nothing} @0x000000001cbaf81a, Ptr{N

Task (runnable) @0x000000000f2ff840

In [6]:
istaskdone(server_task), istaskdone(client)

┌ Error: WebSocketServer Error
│   exception = (HTTP.WebSockets.WebSocketError(HTTP.WebSockets.CloseFrameBody(1008, "Unexpected client websocket error")), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000001cbde16a, Ptr{Nothing} @0x000000001cbdef66, Ptr{Nothing} @0x000000001cbdfddb, Ptr{Nothing} @0x000000001cbdfddb, Ptr{Nothing} @0x000000001cbdfddb, Ptr{Nothing} @0x000000001cbdfddb, Ptr{Nothing} @0x000000001cbdfddb, Ptr{Nothing} @0x000000001cbdfddb, Ptr{Nothing} @0x000000001cbf9188, Ptr{Nothing} @0x000000001cbf9438, Ptr{Nothing} @0x000000001cbfaa09, Ptr{Nothing} @0x000000001cbfcd23, Ptr{Nothing} @0x000000001cbfe127, Ptr{Nothing} @0x000000001cbf4969, Ptr{Nothing} @0x000000001cbf4d41, Ptr{Nothing} @0x000000001cbf4f0b, Ptr{Nothing} @0x000000001cbf3a7b, Ptr{Nothing} @0x000000001cbf485a, Ptr{Nothing} @0x0000000001f20c9f])
└ @ Main In[1]:17
┌ Error: 04af9b32-9c3c-41b0-924c-080a0b8f233b: Unexpected websocket server error
│   (e, catch_backtrace()) = (HTTP.WebSockets.WebSocketErro

(true, true)

Ping Pong 疎通確認タスクを作成したが、上手く終了を検知できない

そのため、定期的に WebSocket サーバに接続するタスクを別途作成するほうが確実かもしれない

In [1]:
using HTTP, LogRoller, Logging, Sockets

"""
    createWebSocketServer(; hostname::String = "127.0.0.1", port::UInt16 = UInt16(8081))
        -> (server::IOServer, task::Task)

Create WebSocket Server.
"""
createWebSocketServer(; hostname::String = "127.0.0.1", port::UInt16 = UInt16(8081)) = begin
    server = Sockets.listen(Sockets.InetAddr(parse(IPAddr, hostname), port))
    task = @async HTTP.WebSockets.listen(hostname, port; server = server) do ws
        try
            for msg in ws
                HTTP.WebSockets.send(ws, msg)
            end
        catch e
            @error "WebSocketServer Error" exception=(e, catch_backtrace())
            rethrow() # プロセスを終了するために rethrow する
        end
    end
    server, task
end

"""
    createWebSocketClient(endpoint::String)
        -> (task::Task, in_channel::Channel, out_channel::Channel)

Create WebSocket Client.

- channels:
    - in_channel:
        - type: `String`
        - size: 1
    - out_channel:
        - type `Any`
        - size: 1
- events:
    - in_channel: `exit`
        - Terminate the websocket client.
    - in_channel: `send:{any_string_data}`
        - Send `any_string_data` to the websocket server.
        - Response data will be written in out_channel: `Response: {any_string_data}`
"""
createWebSocketClient(endpoint::String) = begin
    c_in = Channel{String}(1)
    c_out = Channel(1)
    task = @async HTTP.WebSockets.open(endpoint) do ws
        try
            while !HTTP.WebSockets.isclosed(ws)
                command = take!(c_in) # wait unitl command input
                commands = string.(split(command, ":", keepempty=false))

                @debug commands

                if commands[1] === "exit"
                    @info "Terminate websocket client..."
                    break
                elseif commands[1] === "send"
                    HTTP.WebSockets.send(ws, commands[2])
                    response = HTTP.WebSockets.receive(ws)
                    put!(c_out, string("Response: ", response))
                elseif commands[1] === "exec"
                    f = Symbol(commands[2])
                    output = @eval $f($ws)
                    @debug output
                    put!(c_out, output)
                end
            end
        catch e
            @error "WebSocketClient Error" exception=(e, catch_backtrace())
            showerror(stderr, e, catch_backtrace())
            rethrow()
        end
    end
    task, c_in, c_out
end

"""
    createWebSocketMonitor(endpoint::String)
        -> (task::Task, out_channel::Channel)

Create task for monitoring WebSocket server.

- channels:
    - out_channel:
        - type `Any`
        - size: 1
- events:
    - When: The connection to endpoint is failed.
        - Thrown error will be written in out_channel.
"""
createWebSocketMonitor(endpoint::String) = begin
    c_out = Channel(1)
    task = @async try
        while true
            sleep(1) # 毎秒実行
            HTTP.WebSockets.open(endpoint) do ws
                ping(ws)
                pong(ws)
            end
        end
    catch e
        # エラーが発生したら out_channel に書き込み
        push!(c_out, e)
        rethrow()
    end
    task, c_out
end

createWebSocketMonitor

In [2]:
server, server_task = createWebSocketServer()
client, c_in, c_out = createWebSocketClient("ws://127.0.0.1:8081")
monitor, c_monitor = createWebSocketMonitor("ws://127.0.0.1:8081")
istaskdone(server_task), istaskdone(client), istaskdone(monitor)

(false, false, false)

┌ Error: eb740fe1-3626-48d4-9c98-f22c4e02bcea: error
│   (e, catch_backtrace()) = (UndefVarError(:ping), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000000001d84d41, Ptr{Nothing} @0x0000000001e18d2f, Ptr{Nothing} @0x000000001cb4ae9f, Ptr{Nothing} @0x000000001cb4bcfc, Ptr{Nothing} @0x000000001cb4da4a, Ptr{Nothing} @0x000000001cb4e9f9, Ptr{Nothing} @0x000000001cb4f01e, Ptr{Nothing} @0x000000001cb4f855, Ptr{Nothing} @0x000000001cb50259, Ptr{Nothing} @0x000000001cb476d5, Ptr{Nothing} @0x000000001cb4831d, Ptr{Nothing} @0x000000001cb48cd3, Ptr{Nothing} @0x000000001cb49a05, Ptr{Nothing} @0x000000001cb4a665, Ptr{Nothing} @0x000000001cb4abe8, Ptr{Nothing} @0x000000001cb4ad98, Ptr{Nothing} @0x000000001cb4ae04, Ptr{Nothing} @0x000000001caf1d4a, Ptr{Nothing} @0x000000001cb445df, Ptr{Nothing} @0x000000001cb47133, Ptr{Nothing} @0x000000001cb47353, Ptr{Nothing} @0x000000001cb473d5, Ptr{Nothing} @0x000000001cb44121, Ptr{Nothing} @0x000000001cb441b5, Ptr{Nothing} @0x000000001cb43f2f, Ptr{N

In [3]:
# Close WebSocket server.
close(server)

sleep(3)

istaskdone(server_task), istaskdone(client), istaskdone(monitor)

┌ Error: WebSocketServer Error
│   exception = (HTTP.WebSockets.WebSocketError(HTTP.WebSockets.CloseFrameBody(1008, "Unexpected client websocket error")), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000001cb1a46a, Ptr{Nothing} @0x000000001cb1b266, Ptr{Nothing} @0x000000001cb36348, Ptr{Nothing} @0x000000001cb3652d, Ptr{Nothing} @0x000000001cb38409, Ptr{Nothing} @0x000000001cb3c838, Ptr{Nothing} @0x000000001cb3dc47, Ptr{Nothing} @0x000000001cb34479, Ptr{Nothing} @0x000000001cb34851, Ptr{Nothing} @0x000000001cb34a1b, Ptr{Nothing} @0x000000001cb3358b, Ptr{Nothing} @0x000000001cb3436a, Ptr{Nothing} @0x0000000001e20c9f])
└ @ Main In[1]:17
┌ Error: d6bcc231-085e-4fb9-9c67-dd258701aee4: Unexpected websocket server error
│   (e, catch_backtrace()) = (HTTP.WebSockets.WebSocketError(HTTP.WebSockets.CloseFrameBody(1008, "Unexpected client websocket error")), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000001cb1a46a, Ptr{Nothing} @0x000000001cb1b266, Ptr{Nothing} @0x00

(true, false, true)

In [5]:
# モニタリング結果にエラーが格納されているか確認
## エラーが格納されているのであれば client タスクを kill
if !isempty(c_monitor.data)
    @async Base.throwto(client, InterruptException())
end

┌ Error: WebSocketClient Error
│   exception = (InterruptException(), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00000000105815c3, Ptr{Nothing} @0x00000000105817c7, Ptr{Nothing} @0x00000000105819d5, Ptr{Nothing} @0x000000001cb16eac, Ptr{Nothing} @0x000000001cb1cd01, Ptr{Nothing} @0x000000001cb21f2a, Ptr{Nothing} @0x000000001cb26bd1, Ptr{Nothing} @0x000000001cb27b79, Ptr{Nothing} @0x000000001cb28215, Ptr{Nothing} @0x000000001cb28a6f, Ptr{Nothing} @0x000000001cb29479, Ptr{Nothing} @0x000000001cb010c8, Ptr{Nothing} @0x000000001cb02742, Ptr{Nothing} @0x000000001cb037c1, Ptr{Nothing} @0x000000001cb0f74b, Ptr{Nothing} @0x000000001cb104df, Ptr{Nothing} @0x000000001cb10a9e, Ptr{Nothing} @0x000000001cb10c71, Ptr{Nothing} @0x000000001cb10ce4, Ptr{Nothing} @0x000000001caf1d4a, Ptr{Nothing} @0x000000001caf1ed8, Ptr{Nothing} @0x000000001caecc9f, Ptr{Nothing} @0x000000001caefc2a, Ptr{Nothing} @0x000000001caefe79, Ptr{Nothing} @0x000000001caeff05, Ptr{Nothing} @0x000000001cadf386, Ptr{No

Task (runnable) @0x000000000bc8d560

InterruptException:
Stacktrace:
 

 [1] try_yieldto(undo::typeof(Base.ensure_rescheduled))
    @ Base .\task.jl:777
  [2] wait()
    @ Base .\task.jl:837
  [3] wait(c::Base.GenericCondition{ReentrantLock})
    @ Base .\condition.jl:123
  [4] take_buffered(c::Channel{String})
    @ Base .\channels.jl:389
  [5] take!
    @ .\channels.jl:383 [inlined]
  [6] (::var"#7#9"{Channel{Any}, Channel{String}})(ws::HTTP.WebSockets.WebSocket)
    @ Main .\In[1]:50
  [7] (::HTTP.WebSockets.var"#5#6"{Bool, Int64, Int64, var"#7#9"{Channel{Any}, Channel{String}}, String})(http::HTTP.Streams.Stream{HTTP.Messages.Response, HTTP.ConnectionPool.Connection})
    @ HTTP.WebSockets C:\Users\user\.julia\packages\HTTP\GGNaX\src\WebSockets.jl:372
  [8] macro expansion
    @ C:\Users\user\.julia\packages\HTTP\GGNaX\src\clientlayers\StreamRequest.jl:49 [inlined]
  [9] macro expansion
    @ .\task.jl:399 [inlined]
 [10] streamlayer(stream::HTTP.Streams.Stream{HTTP.Messages.Response, HTTP.ConnectionPool.Connection}; iofunction::HTTP.WebSockets.var"#5#

In [6]:
istaskdone(server_task), istaskdone(client), istaskdone(monitor)

(req::HTTP.Messages.Request; iofunction::Function, kw::Base.Pairs{Symbol, Bool, Tuple{Symbol}, NamedTuple{(:verbose,), Tuple{Bool}}})
    @ HTTP.DefaultHeadersRequest C:\Users\user\.julia\packages\HTTP\GGNaX\src\clientlayers\DefaultHeadersRequest.jl:42
 [21] (::HTTP.RedirectRequest.var"#1#4"{HTTP.RedirectRequest.var"#1#2#5"{HTTP.DefaultHeadersRequest.var"#1#3"{HTTP.DefaultHeadersRequest.var"#1#2#4"{HTTP.BasicAuthRequest.var"#1#3"{HTTP.BasicAuthRequest.var"#1#2#4"{HTTP.ContentTypeDetection.var"#1#3"{HTTP.ContentTypeDetection.var"#1#2#4"{HTTP.CookieRequest.var"#1#5"{HTTP.CookieRequest.var"#1#2#6"{HTTP.RetryRequest.var"#1#4"{HTTP.RetryRequest.var"#1#2#5"{HTTP.CanonicalizeRequest.var"#1#3"{HTTP.CanonicalizeRequest.var"#1#2#4"{HTTP.ConnectionRequest.var"#1#4"{HTTP.ConnectionRequest.var"#1#2#5"{HTTP.TimeoutRequest.var"#1#4"{HTTP.TimeoutRequest.var"#1#2#5"{HTTP.ExceptionRequest.var"#1#3"{HTTP.ExceptionRequest.var"#1#2#4"{typeof(HTTP.StreamRequest.streamlayer)}}}}}}}}}}}}}}}}}}}})(req::HTTP.Me

(true, true, true)